In [1]:
import webrtcvad
from pynput import keyboard
import pyaudio
import collections
import tensorflow as tf
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

2025-03-03 15:21:08.719509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741004468.730489   21620 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741004468.733904   21620 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 15:21:08.745507: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Настройки VAD
vad = webrtcvad.Vad()
vad.set_mode(3)  # Агрессивный режим

In [3]:
# Настройки аудио
RATE_ORIG = 48000  # Исходная частота дискретизации
TARGET_RATE = 16000  # Целевая частота дискретизации
CHUNK = 960  # Длина фрейма (30 мс для 16000 Гц)
FORMAT = pyaudio.paInt16  # 16-битный формат
CHANNELS = 1  # Моно
audio_buffer = collections.deque(maxlen=int(RATE_ORIG * 1.5))  # Буфер на 1.5 секунды

In [4]:
def is_speech(audio_data):
    if isinstance(audio_data, np.ndarray):
        audio_data = audio_data.astype(np.int16).tobytes()  # Преобразование массива NumPy в формат байтов (int16)
    return vad.is_speech(audio_data, sample_rate=RATE_ORIG) # Использование WebRTC VAD для определения наличия речи

In [5]:
def save_audio(waveform, sample_rate, filename="output.wav"):
    waveform_int16 = np.int16(waveform * 32767) # Преобразование значений аудиосигнала из диапазона [-1, 1] в 16-битный формат PCM
    scipy.io.wavfile.write(filename, sample_rate, waveform_int16) # Сохранение аудиофайла в формате WAV

In [6]:
def stream_audio(model, commands):
    p = pyaudio.PyAudio() # Инициализация PyAudio
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE_ORIG, input=True, frames_per_buffer=CHUNK)
    print("Ожидается команда...")
    
    try:
        while True:
            audio_data = stream.read(CHUNK, exception_on_overflow=False) # Чтение аудиофрагмента
            
            # Чтение аудиофрагмента (CHUNK = 960 выборок, что соответствует 30 мс при 16 кГц)
            audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0
            audio_buffer.extend(audio_np)  # Добавление данных в буфер (1.5 секунды)

             # Проверка наличия речи и достаточного количества данных для обработки
            print(is_speech(audio_data))
            if is_speech(audio_data) and len(audio_buffer) >= RATE_ORIG:
                print(1)
                # Преобразование буфера в массив NumPy
                waveform_orig = np.array(audio_buffer, dtype=np.float32)

                # Изменение частоты дискретизации
                waveform_resampled = librosa.resample(waveform_orig, orig_sr=RATE_ORIG, target_sr=TARGET_RATE)
                # save_audio(waveform_resampled, TARGET_RATE, "denoised_output.wav")

                # Преобразование массива в тензор TensorFlow
                waveform_tensor = tf.convert_to_tensor(waveform_resampled, dtype=tf.float32)

                # Распознавание команды и получение уверенности модели
                command_id, confidence = predict_command(model, waveform_tensor)
                if confidence > 0.7:
                    print(f"Распознано: {commands[command_id]}")
                    
                audio_buffer.clear() # Очистка буфера после обработки
    
    except KeyboardInterrupt:
        # Обработка прерывания (Ctrl+C)
        print("\nОстановка...")
        stream.stop_stream()
        stream.close()
        p.terminate()

In [20]:
model = tf.keras.models.load_model('SavedModels/14class_50epochs_base_model.keras')
commands = ['zero', 'up', 'right', 'three', 'on', 'two', 'four', 'yes', 'five' ,'one', 'left', 'down', 'no', 'off']

stream_audio(model, commands)

Ожидается команда...
Распознано: up
Распознано: no
Распознано: up
Распознано: off
Распознано: off
Распознано: on
Распознано: off
Распознано: right
Распознано: five
Распознано: two
Распознано: two
Распознано: right
Распознано: left
Распознано: up
Распознано: right
Распознано: two
Распознано: up
Распознано: left
Распознано: left
Распознано: yes
Распознано: no
Распознано: off
Распознано: no
Распознано: zero
Распознано: yes

Остановка...
